In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
# import urllib2
import shutil
import re
import time
import json
import unicodedata
import html
import urllib

In [5]:
f = open("./data/yogajournal.json")
yoga_json = json.load(f)
urls = []
for pose in yoga_json:
    urls.append(yoga_json[pose]["url"])

In [7]:
fix_chars = ['\x92']
remove_string = 'GO BACK TO A-Z POSE FINDER'

def scrape_page(url, n):
    print("Starting scrape on Page " + str(n))
    URL2 = url
    page = requests.get(URL2)
    soup = BeautifulSoup(page.content, 'html.parser')
    headers = soup.find_all('h3')
    headers_lst = []
    for head in headers:
        pass

    results = soup.find_all('p')
    results_lst = []
    for res in results:
        words  = res.text
        cleaned = html.unescape(words)
        cleaned = unicodedata.normalize("NFKD", cleaned)
        if cleaned == remove_string: # if it is a go back , ignore
            continue
        if "See also " in cleaned: # ignore if it is a see also
            continue
        if "Watch " in cleaned: # ignore if it is a watch
            continue
        if len(cleaned) < 5: # if the string is less than 5 characters ignore
            continue
        results_lst.append(cleaned)
#     print(results_lst)
#     print("#" * 100)
    return results_lst

scrape_dict = dict()
counter = 1
for url in urls:
    results_lst = scrape_page(url, counter)
    counter += 1
    scrape_dict[url] = results_lst
    
for pose in yoga_json:
    url = yoga_json[pose]["url"]
    yoga_json[pose]["description"] = scrape_dict[url]



Starting scrape on Page 1
Starting scrape on Page 2
Starting scrape on Page 3
Starting scrape on Page 4
Starting scrape on Page 5
Starting scrape on Page 6
Starting scrape on Page 7
Starting scrape on Page 8
Starting scrape on Page 9
Starting scrape on Page 10
Starting scrape on Page 11
Starting scrape on Page 12
Starting scrape on Page 13
Starting scrape on Page 14
Starting scrape on Page 15
Starting scrape on Page 16
Starting scrape on Page 17
Starting scrape on Page 18
Starting scrape on Page 19
Starting scrape on Page 20
Starting scrape on Page 21
Starting scrape on Page 22
Starting scrape on Page 23
Starting scrape on Page 24
Starting scrape on Page 25
Starting scrape on Page 26
Starting scrape on Page 27
Starting scrape on Page 28
Starting scrape on Page 29
Starting scrape on Page 30
Starting scrape on Page 31
Starting scrape on Page 32
Starting scrape on Page 33
Starting scrape on Page 34
Starting scrape on Page 35
Starting scrape on Page 36
Starting scrape on Page 37
Starting s

In [8]:
print(yoga_json)

{'Boat Pose': {'body_part': ['abs', 'belly', 'hips', 'liver', 'spine', 'thyroid'], 'url': 'https://www.yogajournal.com/poses/full-boat-pose', 'image_name': 'BoatPose.jpg', 'description': ['Paul Miller', '(par-ee-POOR-nah nah-VAHS-anna)paripurna = full, entire, completenava = boat', 'Step by Step', 'Step 1', ' Sit on the floor with your legs straight in front of you. Press your hands on the floor a little behind your hips, fingers pointing toward the feet, and strengthen the arms. Lift through the top of the sternum and lean back slightly. As you do this make sure your back doesn\'t round; continue to lengthen the front of your torso between the pubis and top sternum. Sit on the "tripod" of your two sitting bones and tailbone.', 'Step 2', " Exhale and bend your knees, then lift your feet off the floor, so that the thighs are angled about 45-50 degrees relative to the floor. Lengthen your tailbone into the floor and lift your pubis toward your navel. If possible, slowly straighten your k

In [13]:
dump_loc = open("./data/cleaner_description_yoga_json.json", "w")
json.dump(yoga_json, dump_loc)
dump_loc.close()

In [28]:
f_des = open("./data/description_yoga_json.json", "r")
description = json.load(f_des)
f_cle = open("./data/cleaner_description_yoga_json.json", "r")
cleaned = json.load(f_cle)

In [31]:
new_description = dict()
for key in description:
    if key not in cleaned:
        new_description[key] = description[key]
    else:
        inner_dict = description[key]
        inner_dict["description"] = cleaned[key]["description"]
        new_description[key] = inner_dict

In [35]:
dump_loc = open("./data/description_yoga_json.json", "w")
json.dump(new_description, dump_loc)
dump_loc.close()

In [44]:
f_des = open("./data/description_yoga_json.json", "r")
description = json.load(f_des)

In [45]:
def organize_description(body):
    
    prologue = [""]
    description = body
    epilogue = [""]
    
    if "Step 1" in body:
        description = []
        step_1_idx = body.index("Step 1")
        description.append(body[step_1_idx]) # step 1
        description.append(body[step_1_idx + 1]) # stuff to do in step 1
        idx = step_1_idx + 2
        while(("Step" in body[idx]) or ((idx + 1) < len(body) and ("Step" in body[idx + 1])) ):
            if "Step" in body[idx + 1]:
                idx += 1 # Ignore mores or for more poses
            description.append(body[idx])
            description.append(body[idx + 1])
            idx += 2
        
        prologue = body[ :step_1_idx] # stuff before steps
        epilogue = body[(idx) : ] # stuff after steps
        
    final_dict = {"introduction" : prologue, "steps" : description, "remarks" : epilogue}
    return final_dict

organized_description = dict()

for key in description:
    body_list = description[key]
    body = description[key]["description"]
    new_body = organize_description(body)
    body_list["organized_output"] = new_body
    organized_description[key] = body_list



    
        

In [46]:
dump_loc = open("./data/organized_output_yoga_json.json", "w")
json.dump(organized_description, dump_loc)
dump_loc.close()

In [3]:
## DOCUMENT 3 http://www.theyogaposes.com/ ###

html_page = urllib.request.urlopen("http://www.theyogaposes.com/")
soup = BeautifulSoup(html_page, "html.parser")
link_set = set()
for link in soup.findAll('a'):
    link_name = link.get('href')
    if "-pose" in link_name:
        link_set.add(link_name)



In [14]:
body_parts = ["knees", "hands", "arms", "legs", "calves", "back", "lower back"
             "ankles", "shoulders", "upper back", "neck", "glutes", "hips", "butt"
             "elbows", "hamstrings", "stomach", "spine"]
extended_dict = dict()
for link_name in link_set:
    try:
        html_page = urllib.request.urlopen(link_name)
    except:
        continue
    soup = BeautifulSoup(html_page, "html.parser")
    
    page_text = []
    
    for para in soup.findAll('p'):
        text = para.text
        if ("Congratulations" not in text):
            continue
        if ("#" in text):
            continue
        page_text.append(text.strip())
        
    pose_idx = link_name.find("=") + 1
    pose_name = link_name[pose_idx:]
    pose_name = pose_name.replace("-", " ")
    pose_name = pose_name.split()
    cap_pos = []
    for name in pose_name:
        cap_name = name[0].upper() + name[1:]
        cap_pos.append(cap_name)
    cap_name = " ".join(cap_pos)
    if "Yoga" == cap_name[:4]:
        cap_name = cap_name[4:]
    cap_name = cap_name.strip()
    
#     print(cap_name)
#     print(link_name)
#     print(text)
#     print("#" * 100)

    description = page_text
    full_text = " ".join(description)
    bps = [bp for bp in body_parts if (bp in full_text)]
    
    if bps == []:
        continue
    print(bps)
    
    inner_dict = dict()
    inner_dict["body_part"] = bps
    inner_dict["url"] = link_name
    inner_dict["image_name"] = "NONE"
    inner_dict["description"] = description
    inner_dict["organized_output"] = {"introduction" : [], "steps" : [], "remarks" : []}
    extended_dict[cap_name] = inner_dict
print(extended_dict)
        


['hands', 'legs', 'back', 'shoulders', 'hips']
['back', 'shoulders', 'spine']
['back', 'neck']
['arms', 'legs', 'back', 'neck']
['knees', 'hands', 'legs', 'back', 'shoulders', 'neck', 'hips', 'spine']
['back', 'stomach']
['hands', 'legs', 'back']
['knees', 'hands', 'back', 'hips', 'spine']
['knees', 'hands', 'arms', 'legs', 'back', 'neck']
['arms', 'back']
['knees', 'hands', 'arms', 'back', 'shoulders']
['knees', 'hands', 'spine']
['knees', 'arms', 'back']
['knees', 'hands', 'back', 'shoulders', 'hips']
['arms', 'back', 'spine']
['hands', 'legs', 'back', 'spine']
['knees', 'back', 'shoulders', 'hips', 'spine']
['knees', 'hands', 'arms', 'legs', 'back', 'neck', 'hips']
['arms', 'hips']
['arms', 'legs', 'back', 'spine']
['arms', 'legs', 'back', 'shoulders', 'spine']
['knees', 'hands', 'back', 'hips']
['knees']
['knees', 'hands', 'arms', 'legs', 'back', 'shoulders', 'upper back', 'neck', 'hips']
['knees', 'hands', 'arms', 'back', 'hips', 'spine']
['knees', 'hands', 'arms', 'legs']
['arms'

In [15]:
dump_loc = open("./data/extended_data.json", "w")
json.dump(extended_dict, dump_loc)
dump_loc.close()

In [2]:
# https://stackoverflow.com/questions/18408307/how-to-extract-and-download-all-images-from-a-website-using-beautifulsoup

import re
import requests
from bs4 import BeautifulSoup

site = "http://www.theyogaposes.com/"

response = requests.get(site)

soup = BeautifulSoup(response.text, 'html.parser')
img_tags = soup.find_all('img')

urls = [img['src'] for img in img_tags]


image_urls = []
for url in urls:
    filename = re.search(r'/([\w_-]+[.](jpg|gif|png))$', url)
    if not filename:
        print("Regex didn't match with the url: {}".format(url))
        continue
    if 'http' not in url:
        # sometimes an image source can be relative 
        # if it is provide the base url which also happens 
        # to be the site variable atm. 
        url = '{}{}'.format(site, url)
    response = requests.get(url)
    print("1")
    print(url)

1
http://www.theyogaposes.com/images/the_yoga_poses.jpg
1
http://www.theyogaposes.com/images/p100p/downward-facing-dog-pose.png
1
http://www.theyogaposes.com/images/p100p/yoga-handstand-pose.png
1
http://www.theyogaposes.com/images/p100p/yoga-firelog-pose.png
1
http://www.theyogaposes.com/images/p100p/akarm-dhanurasana-pose.png
1
http://www.theyogaposes.com/images/p100p/ananda-balasana-pose-100.png
1
http://www.theyogaposes.com/images/p100p/melting-heart-pose-100.png


KeyboardInterrupt: 

In [16]:
f_ext = open("./data/extended_data.json", "r")
extended = json.load(f_ext)
f_organized = open("./data/organized_output_yoga_json.json", "r")
organized = json.load(f_organized)

final = dict()
for key in extended:
    if key not in organized:
        final[key] = extended[key]
for key in organized:
    final[key] = organized[key]
    
print(final)

dump_loc = open("./data/extended_data.json", "w")
json.dump(final, dump_loc)
dump_loc.close()

{'Purvottanasana Pose': {'body_part': ['hands', 'legs', 'back', 'shoulders', 'hips'], 'url': 'http://www.theyogaposes.com/yoga_poses.php?input_page=yoga-purvottanasana-pose', 'image_name': 'NONE', 'description': ['Be certain of your physical abilities and limitations before attempting any yoga pose, and seek clearance from your physician if you are uncertain. Always respect your body and know your limits. Yoga Upward Facing Plank Pose\n\nYoga Purvottanasana Pose\n\n- Inhale, begin seated with legs outstretched.\n\n- Exhale, grounding palms, fingers forward hands beneath shoulders.\n\n- Inhale, drawing in navel to firm core, gounding heels, firming in shoulders.\n\n- Exhale, lifting hips finding alignment from feet through top of head.\n\n- Hold pose 3 to 5 breaths or as long as comfortable.\nPrecautions for Upward Facing Plank Pose - Purvottanasana Pose: Yogis with wrist, elbow, shoulder, hip, knee, ankle or lower back issues should use caution.\n\nCongratulations on learning "Yoga Upw

In [17]:
print(len(organized))
print(len(final))
print(len(extended))

231
271
61
